In [0]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

In [0]:
df = pd.read_csv("drive/My Drive/Data/lyrics_cleaned.csv")

In [0]:
import tensorflow as tf
import gensim
import nltk

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
df = df[['lyrics', 'genre']]

In [9]:
nltk.sent_tokenize(df['lyrics'].values[0])

["Don't tell what you think about this and what i should be I am learning how to live I'll never be the way you are I'm gonna climb this now straight to the top And then we're never gonna stop Destroy the past I'll do everything i can We can not hide our face tonight In all our shame This is our hope for tomorrow It's the only way You don't impress me You think you're everything to everyone But you mean nothing to me now Keep your mouth shut and swallow your pride Because we're alone for the same ride I carry myself all alone It's not too late To make a better change"]

In [0]:
import logging
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [0]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['lyrics'].values)]

In [0]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [13]:
doc2vec_model = Doc2Vec(documents, vector_size=100, window=5, min_count=100, workers=3, epoch=2)

2019-05-10 10:08:00,995 : INFO : collecting all words and their counts
2019-05-10 10:08:00,997 : WARNING : Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
2019-05-10 10:08:01,000 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-05-10 10:08:02,400 : INFO : PROGRESS: at example #10000, processed 11351459 words (8119211/s), 197 word types, 10000 tags
2019-05-10 10:08:03,824 : INFO : PROGRESS: at example #20000, processed 22618697 words (7919261/s), 299 word types, 20000 tags
2019-05-10 10:08:05,225 : INFO : PROGRESS: at example #30000, processed 33765415 words (7961825/s), 299 word types, 30000 tags
2019-05-10 10:08:06,652 : INFO : PROGRESS: at example #40000, processed 45156091 words (7994403/s), 313 word types, 40000 tags
2019-05-10 10:08:08,035 : INFO : PROGRESS: at example #50000, processed 56345133 words (8096058/s), 321 word types, 50000 tags
2019-05-10 10:08:09,444 : INFO : PROG

In [0]:
vector = []

for I in range(len(documents)):
    vector.append([doc2vec_model[I]])
    
vector = np.array(vector)

In [0]:
genre = list(set(df['genre'].values))

In [0]:
one_hot_y = np.array([genre.index(I) for I in df['genre'].values])

In [21]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.13.1


In [0]:
from sklearn.model_selection import KFold

In [0]:
def find_highest(prediction):
  daftar = list(prediction)
  return daftar.index(max(daftar))

In [0]:
def accuracy(pred, real):
  count = 0
  for I in range(len(pred)):
    if pred[I] == real[I]:
      count += 1
  return count / len(pred)

In [25]:
kf = KFold(n_splits = 5)

for train_index, test_index in kf.split(vector):
  X_train, X_test = vector[train_index], vector[test_index]
  y_train, y_test = one_hot_y[train_index], one_hot_y[test_index]
  model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, 100)),  
    keras.layers.Dense(250, activation=tf.nn.relu),
    keras.layers.Dense(250, activation=tf.nn.relu), #relu
    keras.layers.Dense(11, activation=tf.nn.softmax) #softmax output
  ])

  model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  model.fit(X_train, y_train, epochs=5)
  prediction = [find_highest(I) for I in model.predict(X_test)]
  print(accuracy(prediction, y_test))

Epoch 1/5
194092/194092 [==============================] - 20s 105us/sample - loss: 1.5581 - acc: 0.5010
Epoch 2/5
194092/194092 [==============================] - 21s 108us/sample - loss: 1.5090 - acc: 0.5120
Epoch 3/5
194092/194092 [==============================] - 21s 109us/sample - loss: 1.4869 - acc: 0.5182
Epoch 4/5
194092/194092 [==============================] - 20s 103us/sample - loss: 1.4672 - acc: 0.5231
Epoch 5/5
194092/194092 [==============================] - 20s 104us/sample - loss: 1.4493 - acc: 0.5270
0.47997856686519796
Epoch 1/5
194092/194092 [==============================] - 23s 117us/sample - loss: 1.5609 - acc: 0.4994
Epoch 2/5
194092/194092 [==============================] - 21s 109us/sample - loss: 1.5108 - acc: 0.5112
Epoch 3/5
194092/194092 [==============================] - 20s 103us/sample - loss: 1.4876 - acc: 0.5172
Epoch 4/5
194092/194092 [==============================] - 21s 108us/sample - loss: 1.4678 - acc: 0.5216
Epoch 5/5
194092/194092 [==========

In [26]:
kf = KFold(n_splits = 5)

for train_index, test_index in kf.split(vector):
  X_train, X_test = vector[train_index], vector[test_index]
  y_train, y_test = one_hot_y[train_index], one_hot_y[test_index]
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(1, 100)))
  model.add(keras.layers.Reshape((10,10, 1)))
  model.add(keras.layers.Conv2D(8, kernel_size=3, activation='relu'))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(500, activation=tf.nn.sigmoid))
  model.add(keras.layers.Dense(200, activation=tf.nn.sigmoid))
  model.add(keras.layers.Dense(11, activation=tf.nn.softmax))

  model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  model.fit(X_train, y_train, epochs=5)
  prediction = [find_highest(I) for I in model.predict(X_test)]
  print(accuracy(prediction, y_test))

Epoch 1/5
194092/194092 [==============================] - 58s 301us/sample - loss: 1.5808 - acc: 0.4932
Epoch 2/5
194092/194092 [==============================] - 58s 301us/sample - loss: 1.5323 - acc: 0.5054
Epoch 3/5
194092/194092 [==============================] - 56s 289us/sample - loss: 1.5097 - acc: 0.5117
Epoch 4/5
194092/194092 [==============================] - 60s 307us/sample - loss: 1.4903 - acc: 0.5173
Epoch 5/5
194092/194092 [==============================] - 60s 311us/sample - loss: 1.4701 - acc: 0.5236
0.48069987428642086
Epoch 1/5
194092/194092 [==============================] - 58s 299us/sample - loss: 1.5813 - acc: 0.4915
Epoch 2/5
194092/194092 [==============================] - 57s 294us/sample - loss: 1.5316 - acc: 0.5046
Epoch 3/5
194092/194092 [==============================] - 57s 294us/sample - loss: 1.5076 - acc: 0.5120
Epoch 4/5
194092/194092 [==============================] - 57s 295us/sample - loss: 1.4870 - acc: 0.5187
Epoch 5/5
194092/194092 [==========

In [29]:
(0.480 + 0.495 + 0.496 + 0.490 + 0.485) / 5

0.4892

In [39]:
(0.5224 + 0.522 + 0.522 + 0.5242 + 0.5536) / 5

0.52884

In [31]:
kf = KFold(n_splits = 5)

for train_index, test_index in kf.split(vector):
  X_train, X_test = vector[train_index], vector[test_index]
  y_train, y_test = one_hot_y[train_index], one_hot_y[test_index]
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(1, 100)))
  model.add(keras.layers.Reshape((10,10, 1)))
  model.add(keras.layers.Conv2D(4, kernel_size=3, activation='relu'))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(500, activation=tf.nn.sigmoid))
  model.add(keras.layers.Dense(200, activation=tf.nn.sigmoid))
  model.add(keras.layers.Dense(11, activation=tf.nn.softmax))

  model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  model.fit(X_train, y_train, epochs=5)
  prediction = [find_highest(I) for I in model.predict(X_test)]
  print(accuracy(prediction, y_test))

Epoch 1/5
194092/194092 [==============================] - 58s 298us/sample - loss: 1.5888 - acc: 0.4917
Epoch 2/5
194092/194092 [==============================] - 55s 286us/sample - loss: 1.5424 - acc: 0.5026
Epoch 3/5
194092/194092 [==============================] - 52s 269us/sample - loss: 1.5228 - acc: 0.5078
Epoch 4/5
194092/194092 [==============================] - 51s 262us/sample - loss: 1.5063 - acc: 0.5126
Epoch 5/5
194092/194092 [==============================] - 52s 267us/sample - loss: 1.4897 - acc: 0.5177
0.4785359520227521
Epoch 1/5
194092/194092 [==============================] - 60s 309us/sample - loss: 1.5947 - acc: 0.4883
Epoch 2/5
194092/194092 [==============================] - 60s 309us/sample - loss: 1.5446 - acc: 0.5010
Epoch 3/5
194092/194092 [==============================] - 55s 284us/sample - loss: 1.5236 - acc: 0.5073
Epoch 4/5
194092/194092 [==============================] - 56s 289us/sample - loss: 1.5057 - acc: 0.5124
Epoch 5/5
194092/194092 [===========

In [32]:
(0.478 + 0.493 + 0.490 + 0.492 + 0.487) / 5

0.488

In [40]:
(0.5159 + 0.5165 + 0.5163 + 0.5177 + 0.5178) / 5

0.51684

In [33]:
kf = KFold(n_splits = 5)

for train_index, test_index in kf.split(vector):
  X_train, X_test = vector[train_index], vector[test_index]
  y_train, y_test = one_hot_y[train_index], one_hot_y[test_index]
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(1, 100)))
  model.add(keras.layers.Reshape((10,10, 1)))
  model.add(keras.layers.Conv2D(8, kernel_size=3, activation='relu'))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dropout(rate = 0.1))
  model.add(keras.layers.Dense(500, activation=tf.nn.sigmoid))
  model.add(keras.layers.Dense(200, activation=tf.nn.sigmoid))
  model.add(keras.layers.Dense(11, activation=tf.nn.softmax))

  model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  model.fit(X_train, y_train, epochs=5)
  prediction = [find_highest(I) for I in model.predict(X_test)]
  print(accuracy(prediction, y_test))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2019-05-10 12:36:00,729 : WARNING : From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/core.py:143: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Epoch 1/5
194092/194092 [==============================] - 67s 344us/sample - loss: 1.5850 - acc: 0.4905
Epoch 2/5
194092/194092 [==============================] - 66s 341us/sample - loss: 1.5387 - acc: 0.5024
Epoch 3/5
194092/194092 [==============================] - 66s 340us/sample - loss: 1.5176 - acc: 0.5086
Epoch 4/5
194092/194092 [==============================] - 67s 347us/sample - loss: 1.4988 - acc: 0.5147
Epoch 5/5
194092/194092 [==============================] - 67s 346us/sample - loss: 1.4803 - acc: 0.5197
0.4824310120973559
Epoch 1/5
194092/194092 [==============================] - 67s 345us/sample - loss: 1.5855 - acc: 0.4905
Epoch 2/5
194092/194092 [==============================] - 68s 351us/sample - loss: 1.5416 - acc: 0.5022
Epoch 3/5
194092/194092 [==============================] - 65s 334us/sample - loss: 1.5206 - acc: 0.5077
Epoch 4/5
194092/194092 [==============================] - 65s 335us/sample - loss: 1.5027 - acc: 0.5129
Epoch 5/5
194092/194092 [===========

In [34]:
(0.482 + 0.494 + 0.497 + 0.493 + 0.489) / 5

0.49099999999999994

In [41]:
(0.5197 + 0.5184 + 0.5172 + 0.5187 + 0.52) / 5

0.5187999999999999

In [25]:
kf = KFold(n_splits = 5)

for train_index, test_index in kf.split(vector):
  X_train, X_test = vector[train_index], vector[test_index]
  y_train, y_test = one_hot_y[train_index], one_hot_y[test_index]
  model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, 100)),
    keras.layers.Dense(250, activation=tf.nn.relu),
    keras.layers.Dropout(rate = 0.1),
    keras.layers.Dense(250, activation=tf.nn.relu),
    keras.layers.Dense(11, activation=tf.nn.softmax)
  ])

  model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  model.fit(X_train, y_train, epochs=5)
  prediction = [find_highest(I) for I in model.predict(X_test)]
  print(accuracy(prediction, y_test))

Epoch 1/5
194092/194092 [==============================] - 25s 127us/sample - loss: 1.5630 - acc: 0.4993
Epoch 2/5
194092/194092 [==============================] - 23s 120us/sample - loss: 1.5162 - acc: 0.5103
Epoch 3/5
194092/194092 [==============================] - 24s 124us/sample - loss: 1.4956 - acc: 0.5160
Epoch 4/5
194092/194092 [==============================] - 23s 119us/sample - loss: 1.4805 - acc: 0.5196
Epoch 5/5
194092/194092 [==============================] - 23s 119us/sample - loss: 1.4669 - acc: 0.5218
0.4754858520701523
Epoch 1/5
194092/194092 [==============================] - 24s 122us/sample - loss: 1.5642 - acc: 0.4984
Epoch 2/5
194092/194092 [==============================] - 23s 120us/sample - loss: 1.5165 - acc: 0.5102
Epoch 3/5
194092/194092 [==============================] - 23s 121us/sample - loss: 1.4964 - acc: 0.5155
Epoch 4/5
194092/194092 [==============================] - 23s 121us/sample - loss: 1.4802 - acc: 0.5199
Epoch 5/5
194092/194092 [===========

In [42]:
(0.5218 + 0.5299 + 0.5207 + 0.5213 + 0.5218) / 5

0.5231

In [35]:
( 0.475 + 0.490 + 0.499 + 0.490 + 0.499) / 5

0.4906

In [36]:
kf = KFold(n_splits = 5)

for train_index, test_index in kf.split(vector):
  X_train, X_test = vector[train_index], vector[test_index]
  y_train, y_test = one_hot_y[train_index], one_hot_y[test_index]
  model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, 100)),
    keras.layers.Dense(250, activation=tf.nn.relu),
    keras.layers.Dropout(rate = 0.1),
    keras.layers.Dense(250, activation=tf.nn.relu),
      keras.layers.Dropout(rate = 0.1),
    keras.layers.Dense(11, activation=tf.nn.softmax)
  ])

  model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  model.fit(X_train, y_train, epochs=5)
  prediction = [find_highest(I) for I in model.predict(X_test)]
  print(accuracy(prediction, y_test))

Epoch 1/5
194092/194092 [==============================] - 32s 164us/sample - loss: 1.5676 - acc: 0.4984
Epoch 2/5
194092/194092 [==============================] - 28s 145us/sample - loss: 1.5207 - acc: 0.5092
Epoch 3/5
194092/194092 [==============================] - 29s 149us/sample - loss: 1.5010 - acc: 0.5135
Epoch 4/5
194092/194092 [==============================] - 28s 145us/sample - loss: 1.4878 - acc: 0.5175
Epoch 5/5
194092/194092 [==============================] - 28s 144us/sample - loss: 1.4750 - acc: 0.5206
0.4780001236527008
Epoch 1/5
194092/194092 [==============================] - 30s 156us/sample - loss: 1.5703 - acc: 0.4976
Epoch 2/5
194092/194092 [==============================] - 29s 152us/sample - loss: 1.5229 - acc: 0.5090
Epoch 3/5
194092/194092 [==============================] - 28s 145us/sample - loss: 1.5038 - acc: 0.5129
Epoch 4/5
194092/194092 [==============================] - 31s 158us/sample - loss: 1.4894 - acc: 0.5167
Epoch 5/5
194092/194092 [===========

In [37]:
(0.479 + 0.495 + 0.497 + 0.497 + 0.489) / 5

0.49139999999999995

In [43]:
(0.5195 + 0.5192 + 0.5194 + 0.5203 + 0.5206) / 5

0.5198

In [0]:
kf = KFold(n_splits = 5)

for train_index, test_index in kf.split(vector):
  X_train, X_test = vector[train_index], vector[test_index]
  y_train, y_test = one_hot_y[train_index], one_hot_y[test_index]
  model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, 100)),
    keras.layers.Dense(150, activation=tf.nn.relu),
    keras.layers.Dense(150, activation=tf.nn.relu),
    keras.layers.Dense(11, activation=tf.nn.softmax)
  ])

  model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  model.fit(X_train, y_train, epochs=5)
  prediction = [find_highest(I) for I in model.predict(X_test)]
  print(accuracy(prediction, y_test))

Epoch 1/5
194092/194092 [==============================] - 17s 86us/sample - loss: 1.5624 - acc: 0.4991
Epoch 2/5
194092/194092 [==============================] - 17s 88us/sample - loss: 1.5134 - acc: 0.5114
Epoch 3/5
194092/194092 [==============================] - 17s 86us/sample - loss: 1.4940 - acc: 0.5158
Epoch 4/5
194092/194092 [==============================] - 17s 85us/sample - loss: 1.4799 - acc: 0.5189
Epoch 5/5
194092/194092 [==============================] - 16s 85us/sample - loss: 1.4680 - acc: 0.5223
0.4602147435236898
Epoch 1/5
194092/194092 [==============================] - 17s 87us/sample - loss: 1.5659 - acc: 0.4982
Epoch 2/5
194092/194092 [==============================] - 17s 87us/sample - loss: 1.5171 - acc: 0.5098
Epoch 3/5
194092/194092 [==============================] - 18s 94us/sample - loss: 1.4974 - acc: 0.5151
Epoch 4/5
194092/194092 [==============================] - 18s 92us/sample - loss: 1.4830 - acc: 0.5187
Epoch 5/5
194092/194092 [====================

In [0]:
kf = KFold(n_splits = 5)

for train_index, test_index in kf.split(vector):
  X_train, X_test = vector[train_index], vector[test_index]
  y_train, y_test = one_hot_y[train_index], one_hot_y[test_index]
  model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, 100)),
    keras.layers.Dense(500, activation=tf.nn.relu),
    keras.layers.Dense(500, activation=tf.nn.relu),
    keras.layers.Dense(11, activation=tf.nn.softmax)
  ])

  model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  model.fit(X_train, y_train, epochs=5)
  prediction = [find_highest(I) for I in model.predict(X_test)]
  print(accuracy(prediction, y_test))

Epoch 1/5
194092/194092 [==============================] - 51s 265us/sample - loss: 1.5540 - acc: 0.5005
Epoch 2/5
194092/194092 [==============================] - 50s 256us/sample - loss: 1.5024 - acc: 0.5141
Epoch 3/5
194092/194092 [==============================] - 49s 255us/sample - loss: 1.4738 - acc: 0.5204
Epoch 4/5
194092/194092 [==============================] - 49s 254us/sample - loss: 1.4472 - acc: 0.5270
Epoch 5/5
194092/194092 [==============================] - 50s 256us/sample - loss: 1.4156 - acc: 0.5351
0.47371349669229024
Epoch 1/5
194092/194092 [==============================] - 52s 266us/sample - loss: 1.5567 - acc: 0.4994
Epoch 2/5
194092/194092 [==============================] - 51s 265us/sample - loss: 1.5043 - acc: 0.5138
Epoch 3/5
194092/194092 [==============================] - 51s 263us/sample - loss: 1.4752 - acc: 0.5213
Epoch 4/5
194092/194092 [==============================] - 51s 263us/sample - loss: 1.4468 - acc: 0.5282
Epoch 5/5
194092/194092 [==========

In [0]:
kf = KFold(n_splits = 5)

for train_index, test_index in kf.split(vector):
  X_train, X_test = vector[train_index], vector[test_index]
  y_train, y_test = one_hot_y[train_index], one_hot_y[test_index]
  model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, 100)),
    keras.layers.Dense(250, activation=tf.nn.relu),
    keras.layers.Dense(11, activation=tf.nn.softmax)
  ])

  model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  model.fit(X_train, y_train, epochs=5)
  prediction = [find_highest(I) for I in model.predict(X_test)]
  print(accuracy(prediction, y_test))

Epoch 1/5
194092/194092 [==============================] - 14s 73us/sample - loss: 1.5722 - acc: 0.4969
Epoch 2/5
194092/194092 [==============================] - 14s 72us/sample - loss: 1.5289 - acc: 0.5078
Epoch 3/5
194092/194092 [==============================] - 14s 72us/sample - loss: 1.5134 - acc: 0.5111
Epoch 4/5
194092/194092 [==============================] - 14s 73us/sample - loss: 1.5014 - acc: 0.5144
Epoch 5/5
194092/194092 [==============================] - 14s 72us/sample - loss: 1.4916 - acc: 0.5170
0.47507367640088205
Epoch 1/5
194092/194092 [==============================] - 14s 74us/sample - loss: 1.5764 - acc: 0.4955
Epoch 2/5
194092/194092 [==============================] - 14s 74us/sample - loss: 1.5322 - acc: 0.5067
Epoch 3/5
194092/194092 [==============================] - 14s 72us/sample - loss: 1.5153 - acc: 0.5112
Epoch 4/5
194092/194092 [==============================] - 14s 73us/sample - loss: 1.5026 - acc: 0.5143
Epoch 5/5
194092/194092 [===================

In [0]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
lirik = df['lyrics'].values

In [0]:
"Banda".lower()

'banda'

In [0]:
def stopword_removal(teks):
  words = nltk.word_tokenize(teks)
  
  stopwords_list = nltk.corpus.stopwords.words('english')
  
  filtered_words = [word.lower() for word in words if word.lower() not in stopwords_list]
  return filtered_words

In [0]:
filtered_lirik = [stopword_removal(I) for I in lirik]

In [0]:
filtered_lirik = [" ".join(I) for I in filtered_lirik]

In [0]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(filtered_lirik)]

In [0]:
doc2vec_model = Doc2Vec(documents, vector_size=100, window=5, min_count=100, workers=3, epoch=2)

2019-05-06 14:30:32,218 : INFO : collecting all words and their counts
2019-05-06 14:30:32,219 : WARNING : Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
2019-05-06 14:30:32,221 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-05-06 14:30:33,247 : INFO : PROGRESS: at example #10000, processed 7788993 words (7600827/s), 156 word types, 10000 tags
2019-05-06 14:30:34,265 : INFO : PROGRESS: at example #20000, processed 15531672 words (7617684/s), 254 word types, 20000 tags
2019-05-06 14:30:35,251 : INFO : PROGRESS: at example #30000, processed 23167148 words (7750070/s), 254 word types, 30000 tags
2019-05-06 14:30:36,252 : INFO : PROGRESS: at example #40000, processed 30970439 words (7805908/s), 264 word types, 40000 tags
2019-05-06 14:30:37,237 : INFO : PROGRESS: at example #50000, processed 38641197 words (7800131/s), 271 word types, 50000 tags
2019-05-06 14:30:38,241 : INFO : PROGR

In [0]:
vector = []

for I in range(len(documents)):
    vector.append([doc2vec_model[I]])

vector = np.array(vector)

In [0]:
kf = KFold(n_splits = 5)

for train_index, test_index in kf.split(vector):
  X_train, X_test = vector[train_index], vector[test_index]
  y_train, y_test = one_hot_y[train_index], one_hot_y[test_index]
  model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, 100)),
    keras.layers.Dense(250, activation=tf.nn.relu),
    keras.layers.Dense(11, activation=tf.nn.softmax)
  ])

  model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  model.fit(X_train, y_train, epochs=5)
  prediction = [find_highest(I) for I in model.predict(X_test)]
  print(accuracy(prediction, y_test))

Epoch 1/5
194092/194092 [==============================] - 16s 83us/sample - loss: 1.5758 - acc: 0.4969
Epoch 2/5
194092/194092 [==============================] - 15s 76us/sample - loss: 1.5394 - acc: 0.5061
Epoch 3/5
194092/194092 [==============================] - 15s 76us/sample - loss: 1.5263 - acc: 0.5098
Epoch 4/5
194092/194092 [==============================] - 15s 77us/sample - loss: 1.5179 - acc: 0.5115
Epoch 5/5
194092/194092 [==============================] - 15s 76us/sample - loss: 1.5107 - acc: 0.5139
0.4878305133647961
Epoch 1/5
194092/194092 [==============================] - 15s 78us/sample - loss: 1.5775 - acc: 0.4964
Epoch 2/5
194092/194092 [==============================] - 15s 75us/sample - loss: 1.5405 - acc: 0.5064
Epoch 3/5
194092/194092 [==============================] - 15s 79us/sample - loss: 1.5273 - acc: 0.5093
Epoch 4/5
194092/194092 [==============================] - 15s 76us/sample - loss: 1.5190 - acc: 0.5120
Epoch 5/5
194092/194092 [====================

In [0]:
from gensim.models import Word2Vec

In [0]:
model['book']

In [0]:
dictionaryHasilSimpanan = {}

In [0]:
from gensim.test.utils import common_texts

2019-05-06 15:50:43,164 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-05-06 15:50:43,168 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [0]:
tokenized = [nltk.word_tokenize(I) for I in lirik]

In [0]:
model = Word2Vec(tokenized, size=100, window=5, min_count=10, workers=4)

2019-05-06 16:26:08,193 : INFO : collecting all words and their counts
2019-05-06 16:26:08,195 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-06 16:26:08,822 : INFO : PROGRESS: at sentence #10000, processed 2558964 words, keeping 84145 word types
2019-05-06 16:26:09,464 : INFO : PROGRESS: at sentence #20000, processed 5101671 words, keeping 131840 word types
2019-05-06 16:26:10,082 : INFO : PROGRESS: at sentence #30000, processed 7615542 words, keeping 168026 word types
2019-05-06 16:26:10,755 : INFO : PROGRESS: at sentence #40000, processed 10190274 words, keeping 200506 word types
2019-05-06 16:26:11,395 : INFO : PROGRESS: at sentence #50000, processed 12716925 words, keeping 229294 word types
2019-05-06 16:26:12,091 : INFO : PROGRESS: at sentence #60000, processed 15298683 words, keeping 257121 word types
2019-05-06 16:26:12,728 : INFO : PROGRESS: at sentence #70000, processed 17847919 words, keeping 282120 word types
2019-05-06 16:26:13,362 : INF